In [1]:
import re
from multiprocessing import Pool

from bs4 import BeautifulSoup
from bs4.element import NavigableString as n_str

from lxml import etree

from scraping import standard_works, get, process_chapter, standard_works_urls, standard_works_structure

In [2]:
%%time

def process(args):
    url, tags = args
    return process_chapter(get(url), url, tags)

with Pool(50) as p:
    results = p.map(
        process,
        standard_works_urls(standard_works_structure),
    )

len(results)

CPU times: user 371 ms, sys: 325 ms, total: 695 ms
Wall time: 1min 21s


1592

In [3]:
corpus = etree.Element('corpus')

In [4]:
def add(chapter, tags, corpus):
    "Adds a chapter with given tags to the corpus"

    # Collection is created if not already extant
    if not corpus.xpath(f'/corpus/collection[@name="{tags[0]}"]'):
        corpus.append(
            etree.Element('collection', name=tags[0])
        )

    # Book is created if not already extant
    if not corpus.xpath(f'/corpus/collection[@name="{tags[0]}"]/book[@name="{tags[1]}"]'):            
        corpus.xpath(f'/corpus/collection[@name="{tags[0]}"]')[0].append(
            etree.Element('book', name=tags[1])
        )

    # The chapter is appended, and the corpus returned
    corpus.xpath(f'/corpus/collection[@name="{tags[0]}"]/book[@name="{tags[1]}"]')[0].append(
        chapter
    )

    return corpus

In [5]:
for string, tags in results:
    corpus = add(
        etree.fromstring(string),
        tags,
        corpus
    )

In [6]:
errors = [etree.tostring(c) for c in corpus.xpath('//chapter') if b'&' in etree.tostring(c)]
print(len(errors))

104


In [7]:
errors[0]

b'<chapter name="New Testament/Matthew/2" source="https://www.churchofjesuschrist.org/study/scriptures/nt/matt/2"><summary/><verse number="1">1 Now when Jesus was born in Bethlehem of Jud&#195;&#166;a in the days of Herod the king, behold, there came wise men from the east to Jerusalem,</verse><verse number="2">Saying, Where is he that is born King of the Jews? for we have seen his star in the east, and are come to worship him.</verse><verse number="3">When Herod the king had heard these things, he was troubled, and all Jerusalem with him.</verse><verse number="4">And when he had gathered all the chief priests and scribes of the people together, he demanded of them where Christ should be born.</verse><verse number="5">And they said unto him, In Bethlehem of Jud&#195;&#166;a: for thus it is written by the prophet,</verse><verse number="6">And thou Bethlehem, in the land of Juda, art not the least among the princes of Juda: for out of thee shall come a Governor, that shall rule my people

In [2]:
%%time
x = standard_works(50)
len(x.xpath('//chapter'))

AttributeError: Can't pickle local object 'standard_works_generation.<locals>.process'

In [ ]:
with Pool(10) as p:
    

In [5]:
42.7 / 50 * 1592 / 60

22.65946666666667

In [6]:
remaining = [c for c in x.xpath('//chapter') if b'&' in etree.tostring(c)]
len(remaining)

0

In [ ]:
base_url = 'https://www.churchofjesuschrist.org'

standard_works_base_url = base_url + '/study/scriptures'

In [ ]:
xml = standard_work_generation(standard_work_urls(standard_works))

etree.tostring(xml)

In [ ]:
len(list(standard_work_urls(standard_works)))

In [ ]:
first = etree.Element('corpus')
second = etree.Element('collection', this='second')
third = etree.Element('book', that='third')
first.append(second)
second.append(third)

In [ ]:
first = etree.Element('corpus')
first.xpath('/corpus/collection[@name="This is a name"]')

In [ ]:
for i in range(1, 35):
    chapter = process_chapter('/ot', 'deut', i)
    s = etree.tostring(chapter, pretty_print=True)
    print(i, '&' in s.decode('ascii'))

In [ ]:
for work in standard_works:
    for page in get_all_standard_work_pages(work):
        chapter = 
        s = etree.tostring(process_chapter(page), pretty_print=True)
        print(i, '&' in s.decode('ascii'))